## DVCS Tool Prototypes

These will eventually move into some dedicated files.

In [ ]:
import ipywidgets as W, traitlets as T, pathlib as P, time, pprint, ruamel_yaml
from datetime import datetime

In [ ]:
def make_repo_explorer():
    return W.HBox([
        W.Label("History"),
        W.Select(options=["origin", "collaborator1"], description="remote", multiple=False, rows=1),
        W.Select(options=["main", "dev"], description="head", multiple=False, rows=1)
    ])
make_repo_explorer()

In [ ]:
def make_actions():
    return W.VBox([
        W.Label("Actions"), 
        W.HBox([
            W.Label("Save"), 
            W.HTML("<code>+100/-75</code>", description="Δ"),
            W.Text(description="Message", placeholder="Updated value.yaml"),
            W.Button(description="Save", icon="check")
        ]), 
        W.HBox([
            W.Label("Restore"), 
            W.Select(options=["main", "dev"], description="head", multiple=False, rows=1),
            W.SelectionSlider(options=["just now", "5 minutes ago"], description="history", multiple=False, rows=1),
            W.HTML("<code>+100/-75</code>", description="Δ"),
            W.Button(description="Revert", icon="refresh")
        ]),
    ])
make_actions()

In [ ]:
def make_basic_repo_ui(repo):
    watch = W.Checkbox(description="Watch")
    changes = W.HTML()
    message = W.Text(placeholder="Commit message")
    commit = W.Button(description="Commit")
    
    def _do_commit(*args):
        staged = []
        for change in repo.changes:
            for path in ["a_path", "b_path"]:
                if change[path] and change[path] not in staged:
                    repo.stage(change[path])
                    staged += [change[path]]
        repo.commit(message.value or message.placeholder)
    
    commit.on_click(_do_commit)
    
    def change_html(changes):
        if not changes:
            return "<i>Δ0</i>"
        paths = sum([[change["a_path"], change["b_path"]] for change in changes], [])
        paths = sorted(set([p for p in paths if p]))
        return f"""<i title="{", ".join(paths)}">Δ{len(changes)}</i>"""
        
    T.link((watch, "value"), (repo, "watching"))
    T.dlink((repo, "changes"), (changes, "value"), change_html)
    T.dlink((repo, "changes"), (commit, "disabled"), lambda x: not len(x))
    return W.HBox([W.VBox([watch, changes]), W.VBox([message, commit])])